In [1]:
from keras.models import Sequential
from keras.layers import Dense, Recurrent, BatchNormalization, LSTM
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
import time
import os
from sklearn.model_selection import train_test_split


MODEL_DIR = './project/models/'

Using TensorFlow backend.


In [2]:
def print_samples(x, y, n=10):
    higher = len(x)
    print(higher)
    for i in range(n):
        j = np.random.randint(0, higher)
        print(x[j], y[j])

In [4]:
def split_data(x, y, ratio=0.2):
    # converting to numpy array
    x = np.array(x)
    y = np.array(y)
    
    # splitting dataset in training and testing
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = ratio)
    return x_train, y_train, x_test, y_test

def build_model(x_train, y_train, x_test, y_test, hidden_layers=2, neurons=32, epochs=10, plot=True):

    model = Sequential()
    # model.add(Dense(64, input_shape = (1, 10)))
    for i in range(hidden_layers - 1):
        model.add(LSTM(neurons, return_sequences = True, input_shape=(1,10)))
    model.add(LSTM(neurons))
    
    model.add(Dense(10, activation='sigmoid'))

    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
    metrics=['categorical_accuracy'])

    # default batch size = 32
    t0 = time.time()
    history = model.fit(x_train.reshape(len(x_train),1,10), y_train, validation_split=0.15, epochs=epochs, verbose=0) 
    t1 = time.time()
    if plot:
        print(history.history.keys())
        plt.figure(figsize=(15, 5))
        #  "Accuracy"
        plt.subplot(121)
        plt.plot(history.history['acc'])
        plt.plot(history.history['val_acc'])
        plt.title('model accuracy')
        plt.ylabel('accuracy')
        plt.xlabel('epoch')
        plt.legend(['train', 'validation'], loc='upper left')
        # "Loss"
        plt.subplot(122)
        plt.plot(history.history['loss'])
        plt.plot(history.history['val_loss'])
        plt.title('model loss')
        plt.ylabel('loss')
        plt.xlabel('epoch')
        plt.legend(['train', 'validation'], loc='upper left')
        plt.show()
        
#     print('Evaluating model')
    metrics = model.evaluate(x_test.reshape(len(x_test), 1,10), y_test, verbose=0)
#     print('\n')
    with open('./model_cmp_time.txt', 'a+') as model_f:
        model_f.write('hidden layers: {:}, neurons: {:}, epochs: {:}, time: {:}'
                .format(hidden_layers, neurons, epochs, t1-t0))
        for i in range(len(model.metrics_names)):
            model_f.write(" " + str(model.metrics_names[i]) + ": " + str(metrics[i]))
        model_f.write('\n')


In [5]:
layers = [2, 4, 6, 8]
neurons = [4, 8, 16, 32, 64, 128]
epochs = [4, 8, 16, 32, 64]

for target in os.listdir(MODEL_DIR):
    path = os.path.join(MODEL_DIR, target)
    print(target)
    for router in os.listdir(path):
        model = os.path.join(path, router)
        x = []
        y = []
        for dataset in os.listdir(model):
            file = os.path.join(model, dataset)
            with open(file, 'r') as f:
                lines = f.readlines()
            for line in lines:    
                cnt, label = line.split(',')
                cnt = np.array([int(c) for c in cnt.split()[1:]], dtype=np.int)
                label = np.array([int(l) for l in label.split()], dtype=np.int)
                x.append(cnt)
                y.append(label)
        
        x_t, y_t, x_ts, y_ts = split_data(x, y)
        for layer in layers:
            for neuron in neurons:
                for epoch in epochs:
                    build_model(x_t, y_t, x_ts, y_ts, neurons=neuron, hidden_layers=layer, epochs=epoch, plot = False)

        break
    break

r2_172_168_4_2


KeyboardInterrupt: 